In [1]:
!pip show azureml-train-automl


Name: azureml-train-automl
Version: 1.31.0
Summary: automl
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: None
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages
Requires: azureml-train-automl-client, azureml-dataset-runtime, azureml-automl-runtime, azureml-train-automl-runtime, azureml-automl-core
Required-by: azureml-automl-dnn-nlp


In [2]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.31.0 to work with jp-cpet-eastus-research-ml


In [10]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()
# Split the dataset into training and validation subsets
cpet_ds = ws.datasets.get("cpet-cardiac-dataset-selected")
train_ds, test_ds = cpet_ds.random_split(percentage=0.7, seed=123)
print("Data ready!")

Data ready!


In [11]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "jjp581"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


In [12]:

import azureml.train.automl.utilities as automl_utils

for metric in automl_utils.get_primary_metrics('classification'):
    print(metric)

average_precision_score_weighted
accuracy
AUC_weighted
norm_macro_recall
precision_score_weighted


In [15]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(name='Automated ML Experiment Cardiac',
                             task='classification',
                             compute_target=training_cluster,
                             training_data = train_ds,
                             validation_data = test_ds,
                             label_column_name='CardiacLim',
                             iterations=30,
                             primary_metric = 'norm_macro_recall',
                             max_concurrent_iterations=2,
                             featurization='auto',
                             enable_voting_ensemble = False,
                             enable_stack_ensemble = False
                             )

print("Ready for Auto ML run.")

Ready for Auto ML run.


In [16]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'cpet-automl-cardiac-full')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Submitting Auto ML experiment...
Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
cpet-automl-cardiac-full,AutoML_84c699fb-3312-4c0a-ac03-7b5ff3942af3,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
cpet-automl-cardiac-full,AutoML_84c699fb-3312-4c0a-ac03-7b5ff3942af3,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS

{'runId': 'AutoML_84c699fb-3312-4c0a-ac03-7b5ff3942af3',
 'target': 'jjp581',
 'status': 'Completed',
 'startTimeUtc': '2021-07-05T11:58:24.357147Z',
 'endTimeUtc': '2021-07-05T12:22:46.682492Z',
 'properties': {'num_iterations': '30',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'norm_macro_recall',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'jjp581',
  'AMLSettingsJsonString': '{"path":null,"name":"Automated ML Experiment Cardiac","subscription_id":"2ea3a7ac-8926-4694-9e2c-242e69548c78","resource_group":"jp-cpet-research","workspace_name":"jp-cpet-eastus-research-ml","region":"eastus","compute_target":"jjp581","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":30,"primary_metric":"norm_macro_re

In [22]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(name='Automated ML Experiment Cardiac',
                             task='classification',
                             compute_target=training_cluster,
                             training_data = train_ds,
                             validation_data = test_ds,
                             label_column_name='CardiacLim',
                             iterations=40,
                             primary_metric = 'norm_macro_recall',
                             max_concurrent_iterations=4,
                             featurization='auto',
                             enable_voting_ensemble = False,
                             enable_stack_ensemble = False,
                             allowed_models = ['LogisticRegression','DecisionTree'],
                             model_explainability = True
                             )

print("Ready for Auto ML run.")

Ready for Auto ML run.


In [23]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'cpet-automl-cardiac-interpretable')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Submitting Auto ML experiment...
Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
cpet-automl-cardiac-interpretable,AutoML_9d53c1bc-8144-42e9-8cb0-2d73ada0bf77,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
cpet-automl-cardiac-interpretable,AutoML_9d53c1bc-8144-42e9-8cb0-2d73ada0bf77,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS

{'runId': 'AutoML_9d53c1bc-8144-42e9-8cb0-2d73ada0bf77',
 'target': 'jjp581',
 'status': 'Completed',
 'startTimeUtc': '2021-07-05T13:39:14.238651Z',
 'endTimeUtc': '2021-07-05T13:55:51.124805Z',
 'properties': {'num_iterations': '40',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'norm_macro_recall',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'jjp581',
  'AMLSettingsJsonString': '{"path":null,"name":"Automated ML Experiment Cardiac","subscription_id":"2ea3a7ac-8926-4694-9e2c-242e69548c78","resource_group":"jp-cpet-research","workspace_name":"jp-cpet-eastus-research-ml","region":"eastus","compute_target":"jjp581","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":40,"primary_metric":"norm_macro_re